In [2]:
import ifcopenshell
import time

#INPUT:
in_path = r"C:\Users\eibre\OneDrive - Norconsult Group\Documents\VOS-RIBarmering.ifc"
out_path = in_path#[:-4]+"_color.ifc"
final_path = r'X:\nor\oppdrag\Molde2\522\04\52204946\BIM\Innsynsmodell'
run_solibri_optimizer = True
pset_name = 'IFC Armering'
property_name = '11 PosNr'
colours = [
    [255, 255, 0], [0, 255, 255], [255, 0, 255], [0, 0, 255], [255, 153, 0], [255, 0, 0], [0, 204, 255], [153, 204, 0], [255, 102, 0], [255, 0, 255], [128, 0, 0], [128, 128, 0], [0, 128, 0], [0, 128, 128], [0, 0, 128], [128, 0, 128], [255, 128, 128], [255, 255, 153], [51, 153, 102], [153, 204, 255], [255, 153, 0], [51, 102, 255], [51, 153, 102], [255, 204, 153], [255, 255, 0], [0, 0, 255], [153, 153, 255], [51, 204, 204], [128, 0, 128], [0, 128, 128], [128, 128, 0], [255, 128, 128], [51, 102, 255], [255, 153, 204], [255, 204, 153], [128, 0, 128], [0, 128, 0], [255, 204, 0], [51, 102, 255], [51, 102, 255], [0, 255, 0], [153, 153, 255], [255, 255, 0], [255, 0, 255], [51, 204, 204], [51, 102, 255], [204, 204, 255], [51, 102, 255], [204, 255, 204], [255, 0, 0], [0, 0, 255], [51, 153, 102], [255, 255, 0], [255, 153, 0], [153, 204, 255], [255, 102, 0], [204, 153, 255], [204, 204, 255], [255, 153, 204], [51, 51, 153], [255, 255, 0], [0, 0, 255], [153, 153, 255], [51, 204, 204], [128, 0, 128], [0, 128, 128], [128, 128, 0], [255, 128, 128], [51, 102, 255], [255, 153, 204], [255, 0, 0], [0, 0, 255], [51, 153, 102], [255, 255, 0], [255, 153, 0], [153, 204, 255], [255, 102, 0], [204, 153, 255], [204, 204, 255], [255, 153, 204]
]

# Script starts:
t_start = time.perf_counter()

print(f"Opening file at {in_path}")
f = ifcopenshell.open(in_path)

rebars = f.by_type('IfcReinforcingBar')
print(f'The model contains {len(rebars)} rebars')
IfcSweptDiskSolid = f.by_type('IfcSweptDiskSolid')
print(f'The model contains {len(IfcSweptDiskSolid)} IfcSweptDiskSolid')
styleItems = f.by_type('IfcStyledItem')
print(f'The model contains {len(styleItems)} styled items')
styles = f.by_type('IfcSurfaceStyle')
print(f'The model contains {len(styles)} styles:')

for style in styles:
    print(f' * {style.Name}')
def get_propery_value(rebar, pset_name, p_name):
    for pset in rebar.IsDefinedBy:
        if pset.RelatingPropertyDefinition.Name == pset_name:
            for property in pset.RelatingPropertyDefinition.HasProperties:
                if property.Name == p_name:
                    return property.NominalValue[0]

import collections
from collections import defaultdict
rebar_dict = defaultdict(list)
print(f"Grouping rebar by values of property {property_name} in pset {pset_name}")
for k,v in zip([int(get_propery_value(rebar, pset_name, property_name)) for rebar in rebars], rebars):
    rebar_dict[k].append(v)
    
ordered_rebars = collections.OrderedDict(sorted(rebar_dict.items()))
num_groups = len(ordered_rebars)
print(f"Found {num_groups} different groups")

import math
colours = colours*math.ceil(num_groups/len(colours))
colours = colours[:num_groups]
rgb_colours = list()

for rgb in colours:
    rgb_colours.append(f.createIfcColourRgb(None, rgb[0]/255, rgb[1]/255, rgb[2]/255))
    
if(len(styleItems) > 10):
    print("Removing existing IfcStyledItems")
    for styleItem in styleItems:
        f.remove(styleItem)

i=0
print("Creating styled items for rebar color..")
for k, group in ordered_rebars.items():
    colour = rgb_colours[i]
    print(f"Group where {property_name} = {k} contains {len(group)} elements")
    surfaceStyleShading = f.createIfcSurfaceStyleShading()
    surfaceStyleShading.SurfaceColour = colour
    surfaceStyle = f.createIfcSurfaceStyle(colour.Name, "BOTH",(surfaceStyleShading,))
    presStyleAssign = f.createIfcPresentationStyleAssignment((surfaceStyle,))
    i=i+1
    for rebar in group:
        item = rebar.Representation.Representations[0].Items[0]
        f.createIfcStyledItem(item, (presStyleAssign,), None)
        
print(f"Saving file to {out_path}..")
f.write(out_path)
t_end = time.perf_counter()
print(f"File saved, script completed in in {t_end-t_start:0.4f} seconds")
import os
import subprocess
file_name = os.path.basename(out_path)
if (run_solibri_optimizer):
    print("Running Solibri Optimizer..")

    subprocess.check_call([r'C:\Program Files\Solibri\IFCOptimizer\Solibri IFC Optimizer.exe', '-in=' + out_path, '-out=' + final_path, '-ifc', '-force'])

    file_clean_name = final_path+"\\"+file_name
    file_optimized = file_clean_name[:-4]+"_optimized.ifc"
    print(f"Renaming {file_optimized} to {file_clean_name}")
    os.replace(file_optimized, file_clean_name)
    print("Done!")
else:
    subprocess.call(f'copy "{out_path}" "{final_path}\\{file_name}"', shell=True)


Opening file at C:\Users\eibre\OneDrive - Norconsult Group\Documents\VOS-RIBarmering.ifc
The model contains 57024 rebars
The model contains 57373 IfcSweptDiskSolid
The model contains 57528 styled items
The model contains 4 styles:
 * B500NC
 * NO_Metall - Stål - S355
 * Iron B500 NC
 * Steel, Wrought
Grouping rebar by values of property 11 PosNr in pset IFC Armering
Found 99 different groups
Removing existing IfcStyledItems
Creating styled items for rebar color..
Group where 11 PosNr = 1 contains 9340 elements
Group where 11 PosNr = 2 contains 5089 elements
Group where 11 PosNr = 3 contains 4707 elements
Group where 11 PosNr = 4 contains 3389 elements
Group where 11 PosNr = 5 contains 2842 elements
Group where 11 PosNr = 6 contains 1711 elements
Group where 11 PosNr = 7 contains 2162 elements
Group where 11 PosNr = 8 contains 2620 elements
Group where 11 PosNr = 9 contains 1960 elements
Group where 11 PosNr = 10 contains 1624 elements
Group where 11 PosNr = 11 contains 999 elements
Gro

In [2]:
import ifcopenshell
in_path = r"X:\nor\oppdrag\Molde2\522\04\52204946\BIM\Innsynsmodell\VOS-RIBarmering.ifc"
out_path = in_path
f = ifcopenshell.open(in_path)

No stream support: No module named 'lark'


In [23]:
def get_complete_tag(rebar):

    n = ifcopenshell.util.element.get_pset(element=rebar, name="IFC Armering", prop="12 Antall i gruppe")
    ø = ifcopenshell.util.element.get_pset(element=rebar, name="IFC Armering", prop="13 Diameter")
    s = ifcopenshell.util.element.get_pset(element=rebar, name="IFC Armering", prop="14 Senteravstand")
    p1 = ifcopenshell.util.element.get_pset(element=rebar, name="IFC Armering", prop="10 Konstruksjonsdel")
    p2 = ifcopenshell.util.element.get_pset(element=rebar, name="IFC Armering", prop="11 PosNr")
    placement = ifcopenshell.util.element.get_pset(element=rebar, name="IFC Armering", prop="15 Plassering")
    comments = ifcopenshell.util.element.get_pset(element=rebar, name="IFC Armering", prop="16 Kommentar")
    return f'{n}ø{ø}c{s}-{p1}{p2} {placement} {comments}'

rebars = f.by_type('IfcReinforcingBar')
print('The model contains:')
print(f'{len(rebars)} rebars such as {rebars[0].Name}')
print(get_complete_tag(rebars[0]))
IfcReinforcingBarTypes = f.by_type('IfcReinforcingBarType')
print(f'{len(IfcReinforcingBarTypes)} IfcReinforcingBarTypes')
for IfcReinforcingBarType in IfcReinforcingBarTypes:
    print(f' * {IfcReinforcingBarType.Name}')
IfcSweptDiskSolid = f.by_type('IfcSweptDiskSolid')
print(f'{len(IfcSweptDiskSolid)} IfcSweptDiskSolid')
styleItems = f.by_type('IfcStyledItem')
print(f'{len(styleItems)} styled items')

styles = f.by_type('IfcSurfaceStyle')
print(f'{len(styles)} styles:')

for style in styles:
    print(f' * {style.Name}')

import ifcopenshell.util
import ifcopenshell.util.element
print(ifcopenshell.util.element.get_psets(rebars[0]))



The model contains:
56774 rebars such as 126ø12c175-SF1-11 UK På tvers i såle
126.0ø12.0c175.0-SF1-11 UK På tvers i såle
0 IfcReinforcingBarTypes
57123 IfcSweptDiskSolid
57278 styled items
4 styles:
 * B500NC
 * Iron B500 NC
 * NO_Metall - Stål - S355
 * Steel, Wrought
{'IFC Armering': {'01 Monteringsbeskrivelse': '126ø12c175-SF1-11 UK På tvers i såle', '10 Konstruksjonsdel': 'SF1-1', '11 PosNr': '1', '12 Antall i gruppe': 126.0, '13 Diameter': 12.0, '14 Senteravstand': 175.0, '15 Plassering': 'UK', '16 Kommentar': 'På tvers i såle', '17 Formkode': '21', '20 A': 200.0, '21 B': 929.9999999999949, '22 C': 200.0, '34 Kapplengde': 1287.1150383789702, '40 Materiale': 'B500NC', '31 H1': 0.0, '32 H2': 0.0, '45 Dordiameter': 31.999999999999996, '50 Gruppe-ID': '1hAg0QwDj9fxl_Fw_IX2F5', 'id': 467028}}
